In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
import os
import pickle
import re
import konlpy
import nltk
import numpy as np

In [14]:
url = 'https://www.coupang.com/vp/products/6432166864?itemId=13885840489&vendorItemId=3191268334&pickType=COU_PICK&q=SAT%2C%EC%8F%9C%EB%A6%AC%EC%84%9C%EC%B9%98&itemsCount=36&searchId=359d9b926c5c4c94b51af5fc71cf063b&rank=0&isAddedCart='

driver = webdriver.Chrome()
driver.get(url)

In [15]:
driver.find_element(By.XPATH, '//*[@id="btfTab"]/ul[1]/li[2]').click()

In [16]:
reviews = driver.find_elements(By.CSS_SELECTOR, '.js_reviewArticleListContainer article')

In [31]:
review_list = list()

In [32]:
for review in reviews[:3]:
    comment = review.find_element(By.CLASS_NAME, 'sdp-review__article__list__headline').text
    comment += review.find_element(By.CLASS_NAME, 'sdp-review__article__list__review__content').text.replace('\n', ' ')
    review_list.append(comment)
    

In [33]:
review_list

['간을 도와 피로를 줄이는 밀크시슬, 아티쵸크,강황성분최근에 피로도가 높고, 영양제도 이것 저것 챙겨먹기로 하였는데 과연 내가 그것들을 다 감당할 수 있을까하는 의아심을 가지다가 밀크씨슬을 생각하게 되었다. 결국 피로의 원인도 간, 이런 저런 영양제를 흡수 할 수있게 하는 일을 하는 간에게 도움이 되는 것을 몇 년전에도 많이 생각하며 밀크씨슬을 알게 되었었다. 그때는 해외에서 있을 때였는데, 밀크씨슬을 큰 마트등에서 통째로도 팔고, 거칠게 간 것도 팔았다. 그래서 밥에 넣어먹기도 했다. 밀크씨슬은 알갱이가 엄청 딱딱했고. 집에있는 가전으로 부수기도 엄청 부담스러웠었다.  한 동안 간에 특별한 것을 챙겨먹거나 영양제를 챙겨먹지 않은 상황에서, 공신력있는 쏜리서치사에서 밀크씨슬 제품을 한번에 한알씩 2~3회 먹도록 만들어진 것이 있다는 걸 알게 되어 반가왔다.  더군다나 커큐민, 아티초크 추출물도 함유되어 있다하니, 커큐민 많은 강황은 많이 먹으려 해도 누런이에 대한 염려도 있었는데 캡슐에 들어있으니 그냥 목으로 넘기기만 하면 된다니 얼마나 편한가 말이다.  한가지 이번 주문에서 아쉬웠던 점은 배송포장 상태에 대한 것이었다. 덩그러니 약을 얇은 검으틱틱한 비닐에 넣어 보내왔다. 원래 비닐은 주변의 온갖 먼지를 끌어다 붙이는 데 그 안에 약병이 그냥 무방비로 들어가 있어서 경악했다. 쭈글쭈글하고, 배송스티커도 꽤재재했다. 약병을 뽁뽁이로라도 한번 감았다거나, 외곽을 한번 더 보호해주는 박스가 있었다면 그런 상태에서 와도 뭐라 할 수 없지만 쏜리서치의 이 약은 플라스틱 뚜껑만 달랑 씰링을 해놓고 다른 부분은 약병에 붙은 스티커도 부실해 보인다.  어쨌든 약의 효과 아닌가? 나와 합이 잘맞아서 활성 산소 손상으로부터 간을 보호할수 있게 되고 흡수가 용이하게 캡슐로 포장된, S는 실크마린, A는 아티초크 T 커큐민 , 이 성분들이 재효능을 발휘하길 바란다.',
 '간건강에 최고 너무 좋아요 추천드려요!평소 음주를 자주 하는 편이고 잠을 많이 못자는 상황이라 나이도 있고

In [1]:
def clean_korean_documents(documents):
    #텍스트 정제 (HTML 태그 제거)
    for i, document in enumerate(documents):
        document = BeautifulSoup(document, 'html.parser').text 
        documents[i] = document

    #텍스트 정제 (특수기호 제거)
    for i, document in enumerate(documents):
        document = re.sub(r'[^ ㄱ-ㅣ가-힣]', '', document) #특수기호 제거, 정규 표현식
        documents[i] = document
        
    #텍스트 정제 (불용어 제거)
    df = pd.read_csv('https://raw.githubusercontent.com/cranberryai/todak_todak_python/master/machine_learning_text/clean_korean_documents/korean_stopwords.txt', header=None)
    df[0] = df[0].apply(lambda x: x.strip())
    stopwords = df[0].to_numpy()
    nltk.download('punkt')
    for i, document in enumerate(documents):
        clean_words = [] 
        for word in nltk.tokenize.word_tokenize(document): 
            if word not in stopwords: #불용어 제거
                clean_words.append(word)
        documents[i] = ' '.join(clean_words)  
        
    return documents

In [34]:
clean_review = review_list
clean_korean_documents(clean_review)

[nltk_data] Downloading package punkt to C:\Users\hi-
[nltk_data]     sinchon\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


['간 도 피로 줄이다 밀크 시슬 아티 크다 강황 성분 최근 피로 도 높다 영양 제도 챙기다 먹기 하다 과연 감당 하다 있다 하다 의아 심 가다 밀크 슬 하다 되어다 피로 원인 간 이렇다 저런 양제 흡수 하다 있다 하다 하다 간 도움 되다 몇 년전 하다 밀크 슬 되어다 그때 해외 있다 이다 밀크 슬 크다 마트 통째 도 팔고 거치다 간 파다 밥 넣다 먹기 하다 밀크 슬 알갱이 딱딱하다 있다 가전 부수 기도 부담스럽다 하다 동안 간 특별하다 챙기다 먹다 양제 챙기다 먹다 않다 상황 공신 있다 쏘다 치사 밀크 슬 제품 한번 회 먹다 만들어지다 있다 걸 되어다 반갑다 오다 더군다나 커큐민 아티초크 추출 물 함유 되어다 있다 커큐민 많다 강황 먹다 해도 누런이 대한 염려 있다 캡슐 들다 그냥 목 넘기다 하다 되다 얼마나 편하다 한가지 이번 주문 아쉽다 배송 포장 상태 대한 이다 덩 그렇다 약 얇다 검다 틱 비닐 넣다 보내오다 원래 비닐 주변 온갖 먼지 끌다 붙이다 병 그냥 무방비 들어가다 있다 경악 하다 쭈글쭈글하다 배송 스티커 꽤 재재하다 병 뽁뽁이로 한번 감다 외곽 한번 보호 해주다 박스 있다 그렇다 상태 뭐라다 하다 없다 쏘다 리서치 약 플라스틱 뚜껑 달랑 씰링 하다 부분 병 붙다 스티커 부실하다 보이다 어쨌든 약 효과 아니다 나오다 합 잘맞다 활 산소 손상 간 보호 하다 있다 되다 흡수 용이하다 캡슐 포장 되다 늘다 실크 마린 늘다 아티초크 커큐민 성분 재효 능 발휘 하다 바라다',
 '간 건강 최고 좋다 추천 드리다 평소 음주 자주 하다 편이 잠 못자다 상황 나이 있다 하다 양제 관심 많다 편입 니 항상 먹다 있다 제품 쏘다 리서치 캡슐 정 를 재다 하다 제품 워낙 유명하다 제품 쏘다 리서치 믿다 있다 회사 제품 내용물 좋다 밀크 시슬 불리다 실리 마린 간 기능 개선 도와주다 독소 로부터 간 손상 예방 강력하다 항산화작용 하다 고해 피로 회복 크다 역활 하다 간이 좋다 않다 피로 감 쉬다 느끼다 제품 복용 하다 피로 감 덜 느껴지다 뿐 아니다 아티 초코 성분 